In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import pickle
import numpy as np
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.4.1


Function

In [3]:
def get_top_k_items(dataframe, k):    
    """ Create data frame that return top k ratings from each user
    
    Parameters
    ----------
    dataframe : dataframe input (required columns: user_id and prediction)
    k : Number of top k 

    Returns
    -------
    top_k_df : Sorted top k dataframe by user_id with additional rank columns 
        
    """
    
    # Sort dataframe by col_user and (top k) col_rating
    top_k_df = (dataframe.groupby('user_id', as_index=False)
        .apply(lambda x: x.nlargest(k, 'prediction'))
        .reset_index(drop=True))
    
    # Add ranks
    top_k_df["rank"] = top_k_df.groupby('user_id', sort=False).cumcount() + 1
    
    return top_k_df

def merge_ranking_true_pred(test_df, pred_df, k):    
    """ Create dataFrmae of hit counts vs actual relevant items per user number of unique user ids
    
    Parameters
    ----------
    test_df : Real rating dataframe (required columns: user_id and item_id)
    pred_df : Predicted rating dataframe (required columns: user_id, Recommended_artistID and prediction)
    k : Number of top k 

    Returns
    -------
    df_hit : Dataframe contains hits items per user
    df_hit_count : Dataframe contains the number of hits vs actual relevant items per user 
    n_users : Number of common users in both rating_true and rating_pred
        
    """
        
    # Make sure the prediction and true data frames have the same set of users
    common_users = set(test_df['user_id']).intersection(set(pred_df['user_id']))
    test_df_common = test_df[test_df['user_id'].isin(common_users)]
    pred_df_common = pred_df[pred_df['user_id'].isin(common_users)]
    n_users = len(common_users)

    # Create dataframe that contains hits items per user
    df_hit = get_top_k_items(pred_df_common, k)
    df_hit = pd.merge(df_hit, test_df_common, left_on = ['user_id', 'Recommended_artistID'], 
                      right_on = ['user_id','item_id'])[['user_id', 'Recommended_artistID', "rank"]]

    # Count the number of hits vs actual relevant items per user
    df_hit_count = pd.merge(
        df_hit.groupby('user_id', as_index=False)['user_id'].agg({"hit": "count"}),
        test_df_common.groupby('user_id', as_index=False)['user_id'].agg({"actual": "count"}),on='user_id')

    return df_hit, df_hit_count, n_users

def rmse(test_df, pred_df):
    """ Calculates RMSE for a given dataset
    
    Parameters
    ----------
    test_df : Real rating dataframe (required columns: user_id, item_id and rating)
    pred_df : Predicted rating dataframe (required columns: user_id, Recommended_artistID)

    Returns
    -------
    the calculated RMSE
        
    """
    
    # Create joined table to get rating predicted
    rating_true_pred = pd.merge(test_df, pred_df, left_on = ['user_id', 'item_id'], 
                                right_on = ['user_id','Recommended_artistID'])
    return np.sqrt(mean_squared_error(test_df['rating'] , rating_true_pred['prediction']))

def precision_at_k(test_df, pred_df, k):
    """ Calculate precision at k
    
    Parameters
    ----------
    test_df : Real rating dataframe (required columns: user_id, item_id)
    pred_df : Predicted rating dataframe (required columns: user_id, Recommended_artistID)
    k : Number of top k 

    Returns
    -------
    the calculated precision@K
        
    """

    df_hit, df_hit_count, n_users = merge_ranking_true_pred(test_df, pred_df, k)

    if df_hit.shape[0] == 0:
        return 0.0

    return (df_hit_count["hit"] / k).sum() / n_users

def recall_at_k(test_df, pred_df, k):
    """ Calculate recall at k
    
    Parameters
    ----------
    test_df : Real rating dataframe (required columns: user_id, item_id)
    pred_df : Predicted rating dataframe (required columns: user_id, Recommended_artistID)
    k : Number of top k 

    Returns
    -------
    the calculated recall@K
    
    """
        
    df_hit, df_hit_count, n_users = merge_ranking_true_pred(test_df, pred_df, k)

    if df_hit.shape[0] == 0:
        return 0.0

    return (df_hit_count["hit"] / df_hit_count["actual"]).sum() / n_users

def map_at_k(test_df, pred_df, k):    
    """ Calculate mean average precision at k
    
    Parameters
    ----------
    test_df : Real rating dataframe (required columns: user_id, item_id)
    pred_df : Predicted rating dataframe (required columns: user_id, Recommended_artistID and prediction)
    k : Number of top k 

    Returns
    -------
    the calculated MAP@K
    
    """
    
    df_hit, df_hit_count, n_users = merge_ranking_true_pred(test_df, pred_df, k)

    # No match case
    if df_hit.shape[0] == 0:
        return 0.0

    # calculate reciprocal rank of items for each user and sum them up
    df_hit_sorted = df_hit.copy()
    df_hit_sorted["rr"] = (df_hit_sorted.groupby('user_id').cumcount() + 1) / df_hit_sorted["rank"]
    df_hit_sorted = df_hit_sorted.groupby('user_id').agg({"rr": "sum"}).reset_index()

    df_merge = pd.merge(df_hit_sorted, df_hit_count, on = 'user_id')
    
    return (df_merge["rr"] / df_merge["actual"]).sum() / n_users

def ndcg_at_k(test_df, pred_df, k):    
    """ Calculate Calculate Normalized discounted cumulative gain at k
    
    Parameters
    ----------
    test_df : Real rating dataframe (required columns: user_id, item_id)
    pred_df : Predicted rating dataframe (required columns: user_id, Recommended_artistID and prediction)
    k : Number of top k 

    Returns
    -------
    the calculated NDCG@K
    
    """

    df_hit, df_hit_count, n_users = merge_ranking_true_pred(test_df, pred_df, k)

    if df_hit.shape[0] == 0:
        return 0.0

    # calculate discounted gain for hit items
    df_dcg = df_hit.copy()
    # relevance in this case is always 1
    df_dcg["dcg"] = 1 / np.log1p(df_dcg["rank"])
    # sum up discount gained to get discount cumulative gain
    df_dcg = df_dcg.groupby('user_id', as_index=False, sort=False).agg({"dcg": "sum"})
    # calculate ideal discounted cumulative gain
    df_ndcg = pd.merge(df_dcg, df_hit_count, on=['user_id'])
    df_ndcg["idcg"] = df_ndcg["actual"].apply(
        lambda x: sum(1 / np.log1p(range(1, min(x, k) + 1))) )

    # DCG over IDCG is the normalized DCG
    return (df_ndcg["dcg"] / df_ndcg["idcg"]).sum() / n_users

In [4]:
# Import user_taggedartists Data set
music_df = pd.read_csv("/content/drive/My Drive/hetrec2011-lastfm-2k/music_tags_new.csv")
music_df.head()

,userID,artistID,timestamp,tagValue
0,2,52,1238536800000,trip-hop
1,2,63,1238536800000,ambient
2,2,73,1238536800000,chillout
3,2,94,1238536800000,dance
4,2,96,1238536800000,classic rock


In [5]:
# Select only user who listen at least 10 artists
df = music_df.pivot_table(index = 'userID',  values = 'artistID', aggfunc=np.count_nonzero).reset_index().rename(columns = {'artistID':'no_of_artists'})
list_id = list(df[df['no_of_artists'] < 10]['userID'])
music_df_new = music_df[~music_df['userID'].isin(list_id)]
df_new = music_df_new.pivot_table(index = 'userID',  values = 'artistID', aggfunc=np.count_nonzero).reset_index().rename(columns = {'artistID':'no_of_artists'})
print("Minimum number of artists per user", min(df_new['no_of_artists']))
print("New datset contains", len(music_df_new), "rows")

# Sorted timestamp by userid
music_df_new['user_id'] = music_df_new['userID'].astype("category").cat.codes
music_sort = music_df_new.sort_values(by=['userID', 'timestamp']).reset_index(drop=True)
all_users = list(np.sort(music_sort.userID.unique()))
all_items = list(np.sort(music_sort.artistID.unique()))

Minimum number of artists per user 10
New datset contains 83382 rows


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [6]:
# Get last k artists per user
k = 5
last_listen = music_sort.groupby(['userID']).tail(k)

Split train test dataset

In [7]:
def train_test_split(df,n):
    """
    Splits our original data into one test and one training set. 
    The test set is made up of one item for each user. This is our holdout item used to compute Top@K later.
    The training set is the same as our original data but without any of the holdout items.
    Args:
        df (dataframe): Our original data
    Returns:
        df_train (dataframe): All of our data except holdout items
        df_test (dataframe): Only our holdout items.
    """

    # Create two copies of our dataframe that we can modify
    df_test = df.copy(deep=True)
    df_train = df.copy(deep=True)

    # Group by user_id and select only the last n item
    # Test dataframe
    df_test = df_test.groupby(['user_id']).tail(n)
    df_test = df_test[['userID', 'user_id', 'artistID']]

    # Remove the test set from the test set
    mask_test = df_test.index
    df_train = df_train.drop(mask_test)
    df_train = df_train[['userID', 'user_id', 'artistID']]

    return df_train, df_test

k = 5
df_train, df_test = train_test_split(music_sort,k)

In [8]:
# 405 items with cold start problem

df_train['item_id'] = df_train['artistID'].astype("category").cat.codes
item_list = df_train[['artistID', 'item_id']]
item_list = item_list.drop_duplicates()
item_list.to_csv("/content/drive/My Drive/hetrec2011-lastfm-2k/item_list.csv", index=False)
df_test_new = pd.merge(df_test, item_list, how = 'left', left_on= 'artistID', right_on='artistID')
df_test_new.isnull().sum()

userID        0
user_id       0
artistID      0
item_id     405
dtype: int64

In [9]:
# item_id - category coding is in both datasets
print("training set: ", df_train.shape)
print("testing set: ", df_test_new.shape)

training set:  (78627, 4)
testing set:  (4755, 4)


In [10]:
# Create a dictionary contains watched artists for each user
training_dict = df_train.groupby('user_id')['item_id'].apply(lambda x: x.tolist())
training_dict = training_dict.to_dict()

# Function to random select unwatch artists
def artists_choice(user_id, n):
    choice = set(item_list['item_id']) - set(training_dict[user_id])
    rand_artists = np.random.choice(list(choice), n)
    return list(rand_artists)

def get_train_instances(train, training_dict, ratio):
    # Positive instances
    user_train = list(train['user_id'])
    item_train = list(train['item_id'])
    labels = [1] * len(user_train)

    # Negative instances
    for k, v in training_dict.items():
      num_negatives = len(v) * ratio # Define number of negative sampling by given ratio
      user_train.extend(([k] * num_negatives))
      item_train.extend(artists_choice(k, num_negatives))
      labels.extend(([0] * num_negatives))

    return user_train, item_train, labels

In [11]:
def prediction(training_data, model = 'gmf_model', K = 5):
  df = training_data.iloc[:]
  df_items_unique = df['item_id'].unique()
  df_users_unique = df['user_id'].unique()
  batch_size = 128

  top_k_df = pd.DataFrame()
  for users in range(len(df_users_unique)):
    users_df  = df_users_unique[users]
    artists_watched = df[df['user_id'] == users]['item_id'].unique()
    artists_notwatched = list(set(df_items_unique) - set(artists_watched)) # to be used for prediction

    test_user_input = np.repeat(users, len(artists_notwatched)).reshape(-1,1).astype('int64')
    test_item_input = np.array(artists_notwatched).reshape(-1,1).astype('int64')

  
    if model == 'gmf_model':
      pred_test = gmf_model.predict([test_user_input, test_item_input]) #, batch_size = batch_size
    elif model == 'mlp_model':
      pred_test = mlp_model.predict([test_user_input, test_item_input])
    elif model == 'neumf_model':
      pred_test = neumf_model.predict([test_user_input, test_item_input])
    d = {'pred_user_id': list(i[0] for i in test_user_input), 'Recommended_artistID': list(i[0] for i in test_item_input), 
        'prediction': list(i[0] for i in pred_test)}
    recommended_df = pd.DataFrame(data = d)
    top_k_items = recommended_df.sort_values(by='prediction', ascending = False)[:K]
    if users % 500 == 0 and users != 0:
      print("no. of users: ", users + 1)
    
    top_k_df = pd.concat([top_k_df, top_k_items])

  df_hit = pd.merge(df_test_new, top_k_df, how = 'left', left_on = ['user_id', 'item_id'], right_on = ['pred_user_id','Recommended_artistID'] )

  return top_k_df, df_hit, len(df_hit.dropna())

GMF

In [12]:
# HYPERPARAMS

epochs = 100
latent_features = 64
ratio = 5
batch_size_ = 1024

# Graph

tf.keras.backend.clear_session()

user_input = tf.keras.Input(shape=(1,), dtype='int64', name='user_gmf_input')
item_input = tf.keras.Input(shape = (1,), dtype = 'int64', name = 'item_gmf_input')
gmf_u_var = tf.keras.layers.Embedding(len(all_users), latent_features)(user_input)
gmf_i_var = tf.keras.layers.Embedding(len(item_list), latent_features)(item_input)
gmf_user_flatten = tf.keras.layers.Flatten()(gmf_u_var)
gmf_item_flatten = tf.keras.layers.Flatten()(gmf_i_var)
gmf_matrix = tf.keras.layers.multiply([gmf_user_flatten, gmf_item_flatten])
gmf_output = tf.keras.layers.Dense(1, activation = 'relu')(gmf_matrix)

gmf_model = tf.keras.Model([user_input, item_input], gmf_output)
gmf_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'] ) #

gmf_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_gmf_input (InputLayer)     [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_gmf_input (InputLayer)     [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 64)        60864       user_gmf_input[0][0]             
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 64)        752320      item_gmf_input[0][0]             
______________________________________________________________________________________________

In [13]:
for epoch in range(epochs):
  user_input, item_input, labels = get_train_instances(df_train, training_dict, ratio)
  gmf_model.fit([np.array(user_input), np.array(item_input)], np.array(labels),
                batch_size = batch_size_,  epochs=1, shuffle = True ) 
  
  print(epoch + 1)

top_k_df, df_hit, len_df_hit = prediction(df_train, 'gmf_model', K = 5)

# Prediction
top_k_df['user_id'] = top_k_df['pred_user_id']
print("Precision at K", precision_at_k(df_test_new, top_k_df, k))
print("Recall at K", recall_at_k(df_test_new, top_k_df, k))
print("MAP", map_at_k(df_test_new, top_k_df, k))
print("NDCG", ndcg_at_k(df_test_new, top_k_df, k))

461/461 [==============================] - 6s 8ms/step - loss: 0.6824 - accuracy: 0.8334
1
461/461 [==============================] - 4s 9ms/step - loss: 0.4300 - accuracy: 0.8360
2
461/461 [==============================] - 4s 8ms/step - loss: 0.2770 - accuracy: 0.8730
3
461/461 [==============================] - 4s 8ms/step - loss: 0.2041 - accuracy: 0.9091
4
461/461 [==============================] - 4s 8ms/step - loss: 0.1666 - accuracy: 0.9328
5
461/461 [==============================] - 4s 8ms/step - loss: 0.1401 - accuracy: 0.9504
6
461/461 [==============================] - 4s 9ms/step - loss: 0.1213 - accuracy: 0.9619
7
461/461 [==============================] - 4s 9ms/step - loss: 0.1057 - accuracy: 0.9696
8
461/461 [==============================] - 4s 8ms/step - loss: 0.0932 - accuracy: 0.9753
9
461/461 [==============================] - 4s 8ms/step - loss: 0.0830 - accuracy: 0.9795
10
461/461 [==============================] - 4s 8ms/step - loss: 0.0757 - accuracy: 0.9821


In [14]:
# Export weight

gmf_user_embeddings = gmf_model.layers[2].get_weights()[0]
gmf_item_embeddings = gmf_model.layers[3].get_weights()[0]
print(gmf_item_embeddings.shape,gmf_user_embeddings.shape)

with open('/content/drive/My Drive/gmf_item_embedding_neg.pickle', 'wb') as gmf_item:
    pickle.dump(gmf_item_embeddings, gmf_item)
with open('/content/drive/My Drive/gmf_user_embeddings_neg.pickle', 'wb') as gmf_user:
    pickle.dump(gmf_user_embeddings, gmf_user)

(11755, 64) (951, 64)


MLP Model

In [15]:
tf.keras.backend.clear_session()

epochs = 50
latent_dimension = 32
ratio = 3
batch_size_ = 1024

user_input = tf.keras.Input(shape=(1,), dtype='int32', name='user_mlp_input')
item_input = tf.keras.Input(shape = (1,), dtype = 'int32', name = 'item_mlp_input')
user_mlp_embed = tf.keras.layers.Embedding(len(all_users), latent_dimension,
                                           name = 'user_mlp_embed_layer', input_length=None)(user_input) #, input_length =1
item_mlp_embed = tf.keras.layers.Embedding(len(item_list), latent_dimension, 
                                           name = 'item_mlp_embed_layer', input_length=None)(item_input) #, input_length =1
user_mlp_flatten = tf.keras.layers.Flatten(name = 'user_embed_flatten')(user_mlp_embed)
item_mlp_flatten = tf.keras.layers.Flatten(name = 'item_embed_flatten')(item_mlp_embed)

mlp_join = tf.keras.layers.concatenate([user_mlp_flatten, item_mlp_flatten], axis = -1, name = 'mlp_concat_layer')
mlp_flatten = tf.keras.layers.Flatten(name = 'user_concat_layer_flatten')(mlp_join)
#mlp_dropout1 = tf.keras.layers.Dropout(0.3, name = 'mlp_first_dropout')(mlp_flatten, training = True)

mlp_dense1 = tf.keras.layers.Dense(16, activation='relu')(mlp_flatten)
#mlp_dropout2 = tf.keras.layers.Dropout(0.3)(mlp_dense1, training = True)

mlp_dense2 = tf.keras.layers.Dense(8, activation='relu')(mlp_dense1)
#mlp_dropout3 = tf.keras.layers.Dropout(0.3)(mlp_dense2, training = True)

mlp_dense3 = tf.keras.layers.Dense(4, activation='relu')(mlp_dense2)
#mlp_dropout4 = tf.keras.layers.Dropout(0.3)(mlp_dense3, training = True)

mlp_output = tf.keras.layers.Dense(1)(mlp_dense3) # output is a number

mlp_model = tf.keras.Model([user_input, item_input], mlp_output)
mlp_model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) #binary_crossentropy

mlp_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_mlp_input (InputLayer)     [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_mlp_input (InputLayer)     [(None, 1)]          0                                            
__________________________________________________________________________________________________
user_mlp_embed_layer (Embedding (None, 1, 32)        30432       user_mlp_input[0][0]             
__________________________________________________________________________________________________
item_mlp_embed_layer (Embedding (None, 1, 32)        376160      item_mlp_input[0][0]             
______________________________________________________________________________________________

In [16]:
for epoch in range(epochs):

  user_input, item_input, labels = get_train_instances(df_train, training_dict, ratio)
  
  mlp_model.fit([np.array(user_input), np.array(item_input)], np.array(labels),
                batch_size = batch_size_,  epochs=1, shuffle = True ) 
  
  print(epoch + 1)

top_k_df, df_hit, len_df_hit = prediction(df_train, 'mlp_model', K = 5)

# Evaluation
top_k_df['user_id'] = top_k_df['pred_user_id']
print("Precision at K", precision_at_k(df_test_new, top_k_df, k))
print("Recall at K", recall_at_k(df_test_new, top_k_df, k))
print("MAP", map_at_k(df_test_new, top_k_df, k))
print("NDCG", ndcg_at_k(df_test_new, top_k_df, k))

308/308 [==============================] - 3s 6ms/step - loss: 1.1566 - accuracy: 0.7516
1
308/308 [==============================] - 2s 6ms/step - loss: 0.4869 - accuracy: 0.7708
2
308/308 [==============================] - 2s 6ms/step - loss: 0.4368 - accuracy: 0.8019
3
308/308 [==============================] - 2s 6ms/step - loss: 0.4178 - accuracy: 0.8102
4
308/308 [==============================] - 2s 6ms/step - loss: 0.4030 - accuracy: 0.8137
5
308/308 [==============================] - 2s 6ms/step - loss: 0.3845 - accuracy: 0.8208
6
308/308 [==============================] - 2s 6ms/step - loss: 0.3665 - accuracy: 0.8273
7
308/308 [==============================] - 2s 6ms/step - loss: 0.3487 - accuracy: 0.8408
8
308/308 [==============================] - 2s 6ms/step - loss: 0.3285 - accuracy: 0.8506
9
308/308 [==============================] - 2s 6ms/step - loss: 0.3114 - accuracy: 0.8586
10
308/308 [==============================] - 2s 6ms/step - loss: 0.2962 - accuracy: 0.8660


In [17]:
# Export weight

mlp_user_embeddings = mlp_model.layers[2].get_weights()[0]
mlp_item_embeddings = mlp_model.layers[3].get_weights()[0]

mlp_dense0 = mlp_model.layers[8].get_weights()[0]
mlp_dense1 = mlp_model.layers[9].get_weights()[0]
mlp_dense2 = mlp_model.layers[10].get_weights()[0]

with open('/content/drive/My Drive/mlp_dense0_neg.pickle', 'wb') as dense0:
    pickle.dump(mlp_dense0, dense0)
with open('/content/drive/My Drive/mlp_dense1_neg.pickle', 'wb') as dense1:
    pickle.dump(mlp_dense1, dense1)
with open('/content/drive/My Drive/mlp_dense2_neg.pickle', 'wb') as dense2:
    pickle.dump(mlp_dense2, dense2)
with open('/content/drive/My Drive/mlp_user_embeddings_neg.pickle', 'wb') as mlp_user_embed_train:
    pickle.dump(mlp_user_embeddings, mlp_user_embed_train)
with open('/content/drive/My Drive/mlp_item_embeddings_neg.pickle', 'wb') as mlp_item_embed_train:
    pickle.dump(mlp_item_embeddings, mlp_item_embed_train)

NeuMF

In [18]:
with open("/content/drive/My Drive/gmf_user_embeddings_neg.pickle", 'rb') as gmf_user:
  trained_gmf_users = pickle.load(gmf_user)
with open("/content/drive/My Drive/gmf_item_embedding_neg.pickle", 'rb') as gmf_item:
  trained_gmf_items = pickle.load(gmf_item)
with open("/content/drive/My Drive/mlp_user_embeddings_neg.pickle", 'rb') as mlp_user:
  trained_mlp_users = pickle.load(mlp_user)
with open("/content/drive/My Drive/mlp_item_embeddings_neg.pickle", 'rb') as mlp_item:
  trained_mlp_items = pickle.load(mlp_item)
with open("/content/drive/My Drive/mlp_dense0_neg.pickle", 'rb') as dense0:
  trained_mlp_dense0 = pickle.load(dense0)
with open("/content/drive/My Drive/mlp_dense1_neg.pickle", 'rb') as dense1:
  trained_mlp_dense1 = pickle.load(dense1)
with open("/content/drive/My Drive/mlp_dense2_neg.pickle", 'rb') as dense2:
  trained_mlp_dense2 = pickle.load(dense2)

In [21]:
tf.keras.backend.clear_session()

latent_dimension = 32
latent_features = 64

tf.keras.backend.clear_session()

user_input = tf.keras.Input(shape=(1,), dtype='int64', name='user_input')
item_input = tf.keras.Input(shape = (1,), dtype = 'int64', name = 'item_input')

gmf_u_var = tf.keras.layers.Embedding(len(all_users), latent_features, embeddings_initializer = tf.keras.initializers.Constant(trained_gmf_users), 
                                      name = 'user_gmf_embed_layer', trainable = False)(user_input)
gmf_i_var = tf.keras.layers.Embedding(len(item_list), latent_features, embeddings_initializer = tf.keras.initializers.Constant(trained_gmf_items), 
                                      name = 'item_gmf_embed_layer', trainable = False)(item_input)
user_mlp_embed = tf.keras.layers.Embedding(len(all_users), latent_dimension, embeddings_initializer= tf.keras.initializers.Constant(trained_mlp_users), 
                                           trainable = False)(user_input) #, input_length =1
item_mlp_embed = tf.keras.layers.Embedding(len(item_list), latent_dimension, embeddings_initializer= tf.keras.initializers.Constant(trained_mlp_items),
                                           name = 'trained_mlp_items', trainable = False)(item_input) #, input_length =1

gmf_user_flatten = tf.keras.layers.Flatten()(gmf_u_var)
gmf_item_flatten = tf.keras.layers.Flatten()(gmf_i_var)
gmf_matrix = tf.keras.layers.multiply([gmf_user_flatten, gmf_item_flatten])

user_mlp_flatten = tf.keras.layers.Flatten(name = 'user_embed_flatten')(user_mlp_embed)
item_mlp_flatten = tf.keras.layers.Flatten(name = 'item_embed_flatten')(item_mlp_embed)
mlp_join = tf.keras.layers.concatenate([user_mlp_flatten, item_mlp_flatten], axis = -1, name = 'mlp_concat_layer')
mlp_flatten = tf.keras.layers.Flatten(name = 'user_concat_layer_flatten')(mlp_join)

mlp_dense1 = tf.keras.layers.Dense(16, activation='relu', kernel_initializer=tf.keras.initializers.Constant(trained_mlp_dense0), trainable = False)(mlp_flatten) #
mlp_dense2 = tf.keras.layers.Dense(8, activation='relu', kernel_initializer=tf.keras.initializers.Constant(trained_mlp_dense1), trainable = False)(mlp_dense1) #, trainable = False
mlp_dense3 = tf.keras.layers.Dense(4, activation='relu', kernel_initializer=tf.keras.initializers.Constant(trained_mlp_dense2), trainable = False)(mlp_dense2) #, trainable = False
neumf_input = tf.keras.layers.concatenate([gmf_matrix, mlp_dense3])
neumf_output = tf.keras.layers.Dense(1)(neumf_input)

neumf_model = tf.keras.Model([user_input, item_input], neumf_output)
neumf_model.compile(optimizer = 'sgd', loss = 'binary_crossentropy', metrics = ['accuracy'])

neumf_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 1, 32)        30432       user_input[0][0]                 
__________________________________________________________________________________________________
trained_mlp_items (Embedding)   (None, 1, 32)        376160      item_input[0][0]                 
______________________________________________________________________________________________

In [23]:
epochs = 50
ratio = 3

for epoch in range(epochs):

  user_input, item_input, labels = get_train_instances(df_train, training_dict, ratio)
  
  neumf_model.fit([np.array(user_input), np.array(item_input)], np.array(labels),
                batch_size = 1024,  epochs=1, shuffle = True ) 
  
  print(epoch + 1)

top_k_df, df_hit, len_df_hit = prediction(df_train, 'neumf_model', K = 5)
print(len(df_hit.dropna().drop_duplicates()))

# Evaluation
top_k_df['user_id'] = top_k_df['pred_user_id']
print("Precision at K", precision_at_k(df_test_new, top_k_df, k))
print("Recall at K", recall_at_k(df_test_new, top_k_df, k))
print("MAP", map_at_k(df_test_new, top_k_df, k))
print("NDCG", ndcg_at_k(df_test_new, top_k_df, k))

308/308 [==============================] - 1s 3ms/step - loss: 0.0658 - accuracy: 0.9882
1
308/308 [==============================] - 1s 3ms/step - loss: 0.0615 - accuracy: 0.9912
2
308/308 [==============================] - 1s 3ms/step - loss: 0.0588 - accuracy: 0.9912
3
308/308 [==============================] - 1s 3ms/step - loss: 0.0584 - accuracy: 0.9922
4
308/308 [==============================] - 1s 3ms/step - loss: 0.0588 - accuracy: 0.9925
5
308/308 [==============================] - 1s 3ms/step - loss: 0.0563 - accuracy: 0.9927
6
308/308 [==============================] - 1s 3ms/step - loss: 0.0543 - accuracy: 0.9931
7
308/308 [==============================] - 1s 3ms/step - loss: 0.0538 - accuracy: 0.9937
8
308/308 [==============================] - 1s 2ms/step - loss: 0.0522 - accuracy: 0.9940
9
308/308 [==============================] - 1s 3ms/step - loss: 0.0504 - accuracy: 0.9944
10
308/308 [==============================] - 1s 3ms/step - loss: 0.0490 - accuracy: 0.9947
